## Data insights, few visualizations

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 200

In [ ]:
compfin = pd.read_pickle('compfin.pkl')

In [ ]:
compfin.head()

In [ ]:
print(compfin.columns.values)

## Ideas worth exploring

- general description, totals and aggregated statistics
- boxplots, distributions and correlations
- analisys and insights

## Data chraracteristics:

- [dim] carteira_2016 ~ valor_total | has_detailed_information
- [dim] amount of information per institution ~ valor_total | has_detailed_information
- [dim] field of activity
- calculated metrics
- consolidated finances
- detailed finances

In [ ]:
num_org = compfin.groupby(['organizacao']).nome_projeto.count().count()
num_proj = compfin.groupby(['organizacao', 'nome_projeto']).valor_total.count().count()
pro_com_dado = compfin[~compfin.valor_total.isnull()].valor_total.count()

val_tot = compfin['valor_total'].sum()
val_sol = compfin['solicitado_iCSHG'].sum()
val_tot_med = compfin['valor_total'].mean()
val_sol_med = compfin['solicitado_iCSHG'].mean()
num_ben = compfin['beneficiarios'].sum()
num_ben_iCSHG = compfin['beneficiarios_iCSHG'].sum()
# somente 23 tem info de carga horária
hora_per_ben = compfin['carga_horaria'].mean()
med_per_capita = compfin['per_capita'].median()
med_per_capita_hor = compfin['per_capita_por_hora'].median()



print("Número de organizações: %d" % num_org)
print("Número de projetos: %d" % num_proj)
print("Número de projetos com dados de aporte: %d" % pro_com_dado)
print("Valor total dos projetos: R$ %.2f MM" % (val_tot/10**6))
print("O valor solicitado ao iCSHG é de %.2f MM, correspondendo a %d%% do total" % (val_sol/10**6, 100*val_sol/val_tot))
print("Valor médio estimado por projeto atendido: %d Mil" % (val_tot_med/10**3))

print("Valor médio solicitado por projeto atendido: %d Mil" % (val_sol_med/10**3))
print("Número de beneficiados: %d" % num_ben)
print("Número de beneficiados iCSHG: %d" % num_ben_iCSHG)

print("Cada beneficiado recebe em média R$ %d"% (val_tot/num_ben))
print("Em média por beneficiario são investidas %d horas" % hora_per_ben)
# print("O valor mediano por instituição do per capita: %0.2f" % med_per_capita)
print("O valor mediano por instituição do per capita/hr : %0.2f" % med_per_capita_hor)

In [ ]:
# Parece um erro de digitação aqui:
compfin.query("carteira_2016 == 0 and valor_total > 0")

5 maiores projetos

In [ ]:
compfin.groupby(['organizacao','nome_projeto','area']).valor_total.sum().nlargest(5)

5 maiores aportes

In [ ]:
compfin.groupby(['organizacao','nome_projeto','area']).solicitado_iCSHG.sum().nlargest(5)

Aportes por área

In [ ]:
compfin[compfin.valor_total > 0].groupby(['area']).nome_projeto.count().sort_values(ascending = False)

In [ ]:
#avalio a consistencia da marcação has_detailed_info

# valor_total == 0
#compfin[compfin.valor_total.isnull()].has_consolidated_info.value_counts()
compfin[compfin.valor_total.isnull()].has_detailed_info.value_counts()

In [ ]:
print()
print("Informação financeira agregada (MM): nota-se que há alguns grandes players")

fin_agg = compfin[compfin.has_detailed_info][['ativo_2014', 'ativo_2015', 'pl_2014', 'pl_2015',
         'receitas_2014', 'receitas_2015' ,'despesas_2014',
         'despesas_2015']]/1000 # Odd! Too low! Have to multiply by 1000

fin_agg.describe().round(1)

In [ ]:
print()
print("Receita detalhada (MM): nota-se que há alguns grandes players")

rec_det = compfin[compfin.has_detailed_info][['rec_empresas_2015', 'rec_fundacao_instituto_2015', 'rec_pf_2015', 'rec_governo_2015',
       'rec_outros_2015']]/10**6# Odd! Too low! Have to multiply by 

rec_det.describe().round(1)

In [ ]:
print()
print("Despesa detalhada (MM): nota-se que há alguns grandes players")

des_det = compfin[compfin.has_detailed_info][['d_projetos', 'd_gerais_e_admin',
       'd_folha_de_pagamento', 'd_outros']]/10**6 # Odd! Too low! Have to multiply by 

des_det.describe().round(1)

- Vale entender outliers em receita
- explorar razao valor_total/solicitado # Há instituições que recebem muito mais do que outras

#### Boxplots, Distributions and correlations

- aprox: 5 outliers, despesas 2014 tem pouca variação

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("%s, %d"%("Aggregated finances", x.size))
ax = sns.boxplot(data = fin_agg, orient = "h")
ax = sns.swarmplot(data = fin_agg, orient = "h", color=".25")
plt.show()

- receita de fundação e pf com pouca variação, empresas têm maior variação. Receita outros relevante!

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("%s, %d"%("Detailed revenues", x.size))
ax = sns.boxplot(data = rec_det, orient = "h")
ax = sns.swarmplot(data = rec_det, orient = "h", color=".25")
plt.show()

- os gastos com projetos são substancialmente maiores do que os demais tipos
- seria interessante clusterizar por % de tipo de gastos

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("%s, %d"%("Detailed expenses", x.size))
ax = sns.boxplot(data = des_det, orient = "h")
ax = sns.swarmplot(data = des_det, orient = "h", color=".25")
plt.show()

- ainda que haja campos calculados, vale entender as relações que aparecem com força
- por exemplo, parece que surge uma divisão entre governo x solicitado

In [ ]:
corr = compfin[compfin.has_detailed_info].corr()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})